In [1]:
import scipy
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))
from utils import data_utils


def seed_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def load_data_from_mat(path, variable_name):
    mat_content = scipy.io.loadmat(path)
    mat_variable = mat_content[variable_name]
    return np.array(mat_variable)


seed_everything(42)
Fs = 500  # sampling frequency . Set to your value
T = 1200  # total time of the simulation, seconds - set you value, you can
N_EMG = 8  # number of EMG channels
N_joints = 20  # number of decodede angles

path = "../gen_data/v2/"
s = load_data_from_mat(os.path.join(path, "./s.mat"), "s")
s_env = load_data_from_mat(os.path.join(path, "./s_env.mat"), "s_env")
s_env = s_env[:, Fs : -(Fs - 1)]  # Обержем согласно том, как это делается в matlab

t = load_data_from_mat(os.path.join(path, "./t.mat"), "t")
t = t[:, Fs : -(Fs - 1)]  # Обержем согласно том, как это делается в matlab

vel1 = load_data_from_mat(os.path.join(path, "./vel1.mat"), "vel1")
vel2 = load_data_from_mat(os.path.join(path, "./vel2.mat"), "vel2")

In [2]:
data = {"myo": s.T, "angles1": vel1.T, "angles2": vel2.T, "ts": t.T}

# Переходим от частоты 500Гц в частоту 250Гц
data["myo"] = data["myo"][::2]
data["angles1"] = data["angles1"][::2]
data["angles2"] = data["angles2"][::2]
data["ts"] = data["ts"][::2]

# Фильтрация данных
empty = np.empty(data["myo"].shape)  # data["myo"].shape = (4501, 8)

std_range = 6

for electrode in range(data["myo"].shape[1]):
    da = data["myo"][:, electrode]
    dat = da[~np.isnan(da)]

    samp_freq = 250  # Sample frequency (Hz)
    notch_freq = 50.0  # Frequency to be removed from signal (Hz)
    quality_factor = 30.0  # Quality factor

    b_notch, a_notch = scipy.signal.iirnotch(notch_freq, quality_factor, samp_freq)
    freq, h = scipy.signal.freqz(b_notch, a_notch, fs=samp_freq)

    emg_filt = scipy.signal.filtfilt(b_notch, a_notch, dat)
    emg_filt_norm = data_utils.butter_highpass_filter(emg_filt, 10, 125, order=5)
    diff_in_length = len(da) - len(emg_filt_norm)
    emg_filt_norm = np.append(emg_filt_norm, np.zeros(diff_in_length) + np.nan)

    empty[:, electrode] = emg_filt_norm

# Удаление выбросов
cleanest = np.empty(data["myo"].shape)
std_threshold = std_range * np.max(np.nanstd(empty, axis=0))

for electrode in range(data["myo"].shape[1]):
    one_signal = empty[:, electrode]
    one_signal_clean = one_signal[~np.isnan(one_signal)]

    clear_up = np.where(
        one_signal_clean < std_threshold, one_signal_clean, std_threshold
    )
    clear_down = np.where(clear_up > -1 * std_threshold, clear_up, -1 * std_threshold)

    diff_in_length = len(one_signal) - len(clear_down)
    final_signal = np.append(clear_down, np.zeros(diff_in_length) + np.nan)
    cleanest[:, electrode] = final_signal

# MinMax нормализация
maxx = np.nanmax(cleanest)
minn = np.nanmin(cleanest)

# normalize data based on the whole set of signals.
# EMG preproc: normalize -> (-1, 1) range as audio.
emg_min_max = (cleanest - minn) / (maxx - minn)  # (0, 1)
emg_min_max = 2 * emg_min_max - 1

# I do scaling on the whole dataset, but that creates vertical shifts (+- 0.2) in data, so i substract this shift
data_myo = emg_min_max - np.nanmean(emg_min_max, axis=0)
data["myo"] = data_myo

Сгенерированные ЭМГ сигналы

In [3]:
signal = data["myo"][:, :].T
targets = data["angles2"][:, :].T
num_samples = signal.shape[1]

# Подготовка данных
sequence_length = 256
num_sequences = num_samples // sequence_length

# Разделение сигнала и целей на последовательности
input_data = np.array([signal[:, i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])
output_data = np.array([targets[:, i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])

# Преобразование данных в тензоры
input_tensor = torch.tensor(input_data, dtype=torch.float32)
output_tensor = torch.tensor(output_data, dtype=torch.float32)

# Создание полного датасета
dataset = TensorDataset(input_tensor, output_tensor)

# Разделение данных на обучающее и валидационное множество
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset = TensorDataset(input_tensor[:train_size], output_tensor[:train_size])
val_dataset = TensorDataset(input_tensor[train_size:], output_tensor[train_size:])

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

Простой сигнал

In [9]:
# Параметры сигнала
duration = 20 * 60  # 4 минуты в секундах
sampling_rate = 500  # Гц
num_samples = duration * sampling_rate

# Создание многоканального сигнала (8 каналов)
t = np.linspace(0, duration, num_samples, endpoint=False)
signal = np.array([np.sin(2 * np.pi * 5 * t + 0.5 * np.random.randn(*t.shape)) for _ in range(8)])  # 5 Гц - частота синусоиды

# Применение линейного преобразования
amplitude_factor = 2.0
offset = 0.5
transformed_signal = amplitude_factor * signal + offset

# Создание многоканальных целей (20 каналов)
additional_channels = 12  # количество дополнительных каналов
additional_signals = np.array([np.abs(transformed_signal[chan % 8] + 0.1 * np.random.randn(*t.shape)) for chan in range(additional_channels)])
targets = np.vstack((transformed_signal, additional_signals))  # Совмещение для общего числа 20 каналов

# Подготовка данных
sequence_length = 256
num_sequences = num_samples // sequence_length

# Разделение сигнала и целей на последовательности
input_data = np.array([signal[:, i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])
output_data = np.array([targets[:, i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])

# Преобразование данных в тензоры
input_tensor = torch.tensor(input_data, dtype=torch.float32)
output_tensor = torch.tensor(output_data, dtype=torch.float32)

# Создание полного датасета
dataset = TensorDataset(input_tensor, output_tensor)

# Разделение данных на обучающее и валидационное множество
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset = TensorDataset(input_tensor[:train_size], output_tensor[:train_size])
val_dataset = TensorDataset(input_tensor[train_size:], output_tensor[train_size:])

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

# Пример получения данных из train_loader
for emg_batch, angles_batch in train_loader:
    print("Train EMG Batch Shape:", emg_batch.shape)  # Ожидается: (batch_size, 8, 256)
    print(
        "Train Angles Batch Shape:", angles_batch.shape
    )  # Ожидается: (batch_size, 20, 256)
    break  # Выводим только первую итерацию для примера

# Пример получения данных из val_loader
for emg_batch, angles_batch in val_loader:
    print("Test EMG Batch Shape:", emg_batch.shape)  # Ожидается: (batch_size, 8, 256)
    print(
        "Test Angles Batch Shape:", angles_batch.shape
    )  # Ожидается: (batch_size, 20, 256)
    break  # Выводим только первую итерацию для примера

Train EMG Batch Shape: torch.Size([256, 8, 256])
Train Angles Batch Shape: torch.Size([256, 20, 256])
Test EMG Batch Shape: torch.Size([256, 8, 256])
Test Angles Batch Shape: torch.Size([256, 20, 256])


In [ ]:
# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

signal = data["myo"][:, :].T
targets = data["angles2"][:, :].T
num_samples = signal.shape[1]

# # Параметры сигнала
# duration = 20 * 60  # 4 минуты в секундах
# sampling_rate = 500  # Гц
# num_samples = duration * sampling_rate

# # Создание многоканального сигнала (8 каналов)
# t = np.linspace(0, duration, num_samples, endpoint=False)
# signal = np.array([np.sin(2 * np.pi * 5 * t + 0.5 * np.random.randn(*t.shape)) for _ in range(8)])  # 5 Гц - частота синусоиды

# # Применение линейного преобразования
# amplitude_factor = 2.0
# offset = 0.5
# transformed_signal = amplitude_factor * signal + offset

# # Создание многоканальных целей (20 каналов)
# additional_channels = 12  # количество дополнительных каналов
# additional_signals = np.array([np.abs(transformed_signal[chan % 8] + 0.1 * np.random.randn(*t.shape)) for chan in range(additional_channels)])
# targets = np.vstack((transformed_signal, additional_signals))  # Совмещение для общего числа 20 каналов

# Подготовка данных
sequence_length = 256
num_sequences = num_samples // sequence_length

# Разделение сигнала и целей на последовательности
input_data = np.array([signal[:, i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])
output_data = np.array([targets[:, i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])

# Преобразование данных в тензоры
input_tensor = torch.tensor(input_data, dtype=torch.float32)
output_tensor = torch.tensor(output_data, dtype=torch.float32)

# Создание полного датасета
dataset = TensorDataset(input_tensor, output_tensor)

# Разделение данных на обучающее и валидационное множество
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset = TensorDataset(input_tensor[:train_size], output_tensor[:train_size])
val_dataset = TensorDataset(input_tensor[train_size:], output_tensor[train_size:])

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

# Определение модели с Bidirectional LSTM
class BidirectionalLSTMModel(nn.Module):
    def __init__(self, input_size=8, hidden_size=64, num_layers=1, output_size=20):
        super(BidirectionalLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.dense = nn.Linear(hidden_size * 2, output_size)  # *2 для двунаправленной LSTM

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Перемещение размерностей на (batch, seq_length, input_size)
        out, _ = self.lstm(x)
        out = self.dense(out)
        out = out.permute(0, 2, 1)  # Вернуть к (batch, output_size, sequence_length)
        return out

# Создание модели и перенесение ее на устройство
model = BidirectionalLSTMModel().to(device)

# Определение функции потерь и оптимизатора
# criterion = nn.MSELoss().to(device)
criterion = nn.L1Loss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.1)

# Обучение
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Перемещение данных на устройство
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)

    # Валидация
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Перемещение данных на устройство
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(val_loader)

    print(f"Эпоха [{epoch + 1}/{num_epochs}], Потери на тренировке: {train_loss:.4f}, Валидация Потери: {val_loss:.4f}")

print("Обучение завершено.")

Using device: cuda
Эпоха [1/200], Потери на тренировке: 0.1963, Валидация Потери: 0.1944
Эпоха [2/200], Потери на тренировке: 0.1956, Валидация Потери: 0.1937
Эпоха [3/200], Потери на тренировке: 0.1950, Валидация Потери: 0.1930
Эпоха [4/200], Потери на тренировке: 0.1942, Валидация Потери: 0.1924
Эпоха [5/200], Потери на тренировке: 0.1937, Валидация Потери: 0.1919
Эпоха [6/200], Потери на тренировке: 0.1930, Валидация Потери: 0.1914
Эпоха [7/200], Потери на тренировке: 0.1927, Валидация Потери: 0.1909
Эпоха [8/200], Потери на тренировке: 0.1920, Валидация Потери: 0.1906
Эпоха [9/200], Потери на тренировке: 0.1919, Валидация Потери: 0.1902
Эпоха [10/200], Потери на тренировке: 0.1916, Валидация Потери: 0.1899
Эпоха [11/200], Потери на тренировке: 0.1914, Валидация Потери: 0.1897
Эпоха [12/200], Потери на тренировке: 0.1908, Валидация Потери: 0.1895
Эпоха [13/200], Потери на тренировке: 0.1910, Валидация Потери: 0.1893
Эпоха [14/200], Потери на тренировке: 0.1907, Валидация Потери: 0.1

In [135]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

# Параметры сигнала
duration = 4 * 60  # 3 минуты в секундах
sampling_rate = 250  # Гц
num_samples = duration * sampling_rate

# Создание сигнала
t = np.linspace(0, duration, num_samples, endpoint=False)
signal = np.sin(2 * np.pi * 5 * t)  # 5 Гц - частота синусоиды

# Преобразование Фурье
amplitude_factor = 2.0  # увеличиваем амплитуду в 2 раза
offset = 0.5  # добавляем смещение 0.5

# Применение линейного преобразования
transformed_signal = amplitude_factor * signal + offset
targets = np.abs(transformed_signal)

# Подготовка данных
sequence_length = 1000  # Выборка 4 секунд
num_sequences = num_samples // sequence_length

input_data = np.array([signal[i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])
output_data = np.array([targets[i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])

# Преобразование данных в тензоры
input_tensor = torch.tensor(input_data, dtype=torch.float32).unsqueeze(-1)
output_tensor = torch.tensor(output_data, dtype=torch.float32)

# Создание полного датасета
dataset = TensorDataset(input_tensor, output_tensor)

# Разделение данных на обучающее и валидационное множество
train_size = int(0.8 * len(dataset))  # 80% на обучение
val_size = len(dataset) - train_size  # 20% на валидацию

train_dataset = TensorDataset(input_tensor[:train_size], output_tensor[:train_size])
val_dataset = TensorDataset(input_tensor[train_size:], output_tensor[train_size:])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Определение модели
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dense = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        # Применяем полносвязный слой ко всем временным шагам 
        out = self.dense(out)
        return out

model = LSTMModel()

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(-1))
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)

    # Валидация
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(-1))
            val_loss += loss.item()

    val_loss /= len(val_loader)

    print(f"Эпоха [{epoch + 1}/{num_epochs}], Потери на тренировке: {train_loss:.4f}, Валидация Потери: {val_loss:.4f}")

print("Обучение завершено.")

Эпоха [1/10], Потери на тренировке: 2.0062, Валидация Потери: 1.9487
Эпоха [2/10], Потери на тренировке: 1.9297, Валидация Потери: 1.8725
Эпоха [3/10], Потери на тренировке: 1.8533, Валидация Потери: 1.7953
Эпоха [4/10], Потери на тренировке: 1.7755, Валидация Потери: 1.7153
Эпоха [5/10], Потери на тренировке: 1.6944, Валидация Потери: 1.6300
Эпоха [6/10], Потери на тренировке: 1.6071, Валидация Потери: 1.5357
Эпоха [7/10], Потери на тренировке: 1.5098, Валидация Потери: 1.4275
Эпоха [8/10], Потери на тренировке: 1.3967, Валидация Потери: 1.2977
Эпоха [9/10], Потери на тренировке: 1.2596, Валидация Потери: 1.1349
Эпоха [10/10], Потери на тренировке: 1.0855, Валидация Потери: 0.9227
Обучение завершено.


In [61]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

# Параметры сигнала
duration = 20 * 60  # 4 минуты в секундах
sampling_rate = 500  # Гц
num_samples = duration * sampling_rate

# Создание многоканального сигнала (8 каналов)
t = np.linspace(0, duration, num_samples, endpoint=False)
signal = np.array([np.sin(2 * np.pi * 5 * t + 0.5 * np.random.randn(*t.shape)) for _ in range(8)])  # 5 Гц - частота синусоиды

# Применение линейного преобразования
amplitude_factor = 2.0
offset = 0.5
transformed_signal = amplitude_factor * signal + offset

# Создание многоканальных целей (20 каналов)
additional_channels = 12  # количество дополнительных каналов
additional_signals = np.array([np.abs(transformed_signal[chan % 8] + 0.1 * np.random.randn(*t.shape)) for chan in range(additional_channels)])
targets = np.vstack((transformed_signal, additional_signals))  # Совмещение для общего числа 20 каналов

# Подготовка данных
sequence_length = 1000
num_sequences = num_samples // sequence_length

# Разделение сигнала и целей на последовательности
input_data = np.array([signal[:, i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])
output_data = np.array([targets[:, i * sequence_length: (i + 1) * sequence_length] for i in range(num_sequences)])

# Преобразование данных в тензоры
input_tensor = torch.tensor(input_data, dtype=torch.float32)
output_tensor = torch.tensor(output_data, dtype=torch.float32)

# Создание полного датасета
dataset = TensorDataset(input_tensor, output_tensor)

# Разделение данных на обучающее и валидационное множество
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Определение полносвязной модели
class FullyConnectedModel(nn.Module):
    def __init__(self, input_size=sequence_length * 8, hidden_size=128, output_size=sequence_length * 20):
        super(FullyConnectedModel, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.fc_layers(x).view(x.size(0), 20, sequence_length)

model = FullyConnectedModel()

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)

    # Валидация
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(val_loader)

    print(f"Эпоха [{epoch + 1}/{num_epochs}], Потери на тренировке: {train_loss:.4f}, Валидация Потери: {val_loss:.4f}")

print("Обучение завершено.")


Эпоха [1/10], Потери на тренировке: 1.6259, Валидация Потери: 0.7145
Эпоха [2/10], Потери на тренировке: 0.4789, Валидация Потери: 0.3764
Эпоха [3/10], Потери на тренировке: 0.3570, Валидация Потери: 0.3449
Эпоха [4/10], Потери на тренировке: 0.3413, Валидация Потери: 0.3397
Эпоха [5/10], Потери на тренировке: 0.3386, Валидация Потери: 0.3393
Эпоха [6/10], Потери на тренировке: 0.3383, Валидация Потери: 0.3386
Эпоха [7/10], Потери на тренировке: 0.3380, Валидация Потери: 0.3384
Эпоха [8/10], Потери на тренировке: 0.3379, Валидация Потери: 0.3383
Эпоха [9/10], Потери на тренировке: 0.3383, Валидация Потери: 0.3391
Эпоха [10/10], Потери на тренировке: 0.3393, Валидация Потери: 0.3413
Обучение завершено.
